In [1]:
import pandas as pd
import random
import time
import json
from random_user_agent.params import SoftwareName, OperatingSystem
from random_user_agent.user_agent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
# from eunjeon import Mecab
import unicodedata
import pyodbc
import re
import collections
class IG_Bot:
    '''creating a Bot that acts like me.'''
    
    def __init__(self, username, password):
        software_names= [SoftwareName.CHROME.value]
        operating_systems=[OperatingSystem.WINDOWS.value]
        
        #Randomized User ID
        user_agent_rotator= UserAgent(software_name = software_names,
                                     operating_systems=operating_systems,
                                     limit=100)
        user_agent= user_agent_rotator.get_random_user_agent()
        chrome_options=Options()
#         chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sendbox")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument(f'user-agent={user_agent}')
        path='c:\python_temp\chromedriver.exe'
        
        cookies_file_path='c:\IG\cookies.json'
        cookie_websites=['https://instagram.com']
        self.cookies_file_path= cookies_file_path
        self.cookie_websites= cookie_websites
        self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)
        
        self.driver.get('https://instagram.com')
        
        time.sleep(1.5)

        try:
            cookies=json.load(open(self.cookies_file_path,'rb'))
            for website in self.cookie_websites:
                for cookie in cookies:
                    self.driver.add_cookie(cookie)
                self.driver.refresh()
        except Exception as e:
            print(str(e))
       
    def loading(self,xpath):
        '''Wait Loading Until presence'''
        WebDriverWait(self.driver,30).until(EC.presence_of_element_located((By.XPATH,xpath)))
    
    def save_cookies(self):
        '''Save cookie'''
        json.dump(self.driver.get_cookies(),open('c:\IG\cookies.json','w'))
    
    def load_cookies(self):
        '''Load cookies into browser'''
        cookies= json.load(open('cookies.json','rb'))
        for cookie in cookies:
            self.driver.add_cookie(cookie)
    
    def login(self):
        '''Login to Instagram'''
        try:
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(username)
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(password)
            self.loading('//*[@id="loginForm"]/div/div[3]')
            time.sleep(0.5)
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
            self.loading('//*[@id="react-root"]/section/main/div/div/div/div/button')
            self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
            self.loading('/html/body/div[5]/div/div/div/div[3]/button[2]')
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
        except:
            self.loading('/html/body/div[5]/div/div/div/div[3]/button[2]')
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()

'''run the Crawler'''           
class Crawler(IG_Bot):
    global insta_dict
    insta_dict = {'img':[],
                  'id':[],
                  'date': [],
                  'like': [],
                  'text': [],
                  'hashtag':[]}
    def run(self):
        continue_=True
        count=0
        query=input('검색어를 입력하시오. > ') 
        self.driver.get('https://www.instagram.com/explore/tags/'+query)
        
        s_time=time.time()
        try:
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
        except:
            pass
        time.sleep(0.5)
        self.loading('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]')
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()
        while count<2000:
            self.crawl()
            
#Acting like person by random
            time.sleep(random.randrange(0,5))
            try:
                self.driver.find_element_by_css_selector('a._65Bje').click()
            except:
                continue_=False
            count+=1
            time.sleep(2)
            if continue_==False:
                break
        print(len(insta_dict['img']))
        print(len(insta_dict['id']))
        print(len(insta_dict['date']))
        print(len(insta_dict['like']))
        print(len(insta_dict['text']))
        print(len(insta_dict['hashtag']))
        print(insta_dict)
        e_time=time.time()
        (f'{e_time-s_time}초 걸렸습니다.')    
        
            
            
            
    
    def crawl(self):
        self.loading('/html/body/div[6]/div[2]/div/article')

        ## 이미지 수집
        time.sleep(0.8)
        try:       
            srcs=self.driver.find_element_by_css_selector('video.tWeC1')
            src=srcs.get_attribute('poster')
            insta_dirct['img'].append(src)
        except:
            src=self.driver.find_element_by_css_selector('img.FFVAD')
            src=src.get_attribute('src')
            insta_dict['img'].append(src)
            
        ## id 정보 수집
        try:
            info_id = self.driver.find_element_by_css_selector('h2._6lAjh').text
            insta_dict['id'].append(info_id)
        except:
            try:          
                info_id = self.driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
            except:  
                info_id = self.driver.find_element_by_css_selector('div.e1e1d').text
                insta_dict['id'].append(info_id)
            


        ## 시간정보 수집 
        uploadTime=self.driver.find_element_by_css_selector('time._1o9PC')
        time_info=uploadTime.get_attribute('datetime')[:19]
        insta_dict['date'].append(time_info)
        

        ## like 정보 수집
        try:
            check=self.driver.find_element_by_css_selector('span.vcOH2').text
            check=re.sub(r'[^0-9]', '', check)
            insta_dict['like'].append(check)
        except:
            if '가장 먼저' in self.driver.find_element_by_css_selector('div.Nm9Fw').text:
                like=0
            elif '여러 명' in self.driver.find_element_by_css_selector('div.Nm9Fw').text:
                id=[]
                self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div[2]/a[2]').click()
                time.sleep(1)
                while True:
                    last_height =self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                    c=self.driver.find_elements_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a')

                    for i in c:  
                        if i.text not in id:
                            id.append(i.text)     
                        else:
                            pass
                    action = webdriver.common.action_chains.ActionChains(self.driver)
                    el=self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]')
                    action.move_to_element_with_offset(el, 300, 10)
                    action.click()
                    action.perform()
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    time.sleep(1)
                    new_height =self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                    if last_height==new_height:
                        self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[1]/div/div[2]/button').click()
                        break
                like=len(id)
            else:
                like=self.driver.find_element_by_css_selector('a.zV_Nj')
                like = re.sub(r'[^0-9]', '', like.text)
            insta_dict['like'].append(like)
           

                

        ##text 정보수집
        try:
            raw_info = self.driver.find_element_by_css_selector('div.C4VMK').text.split()
            raw_info = raw_info[1:len(raw_info)-1]
            text=[]
            cleantext=[]
            for i in raw_info:
                if '#' in i:
                    pass
                else:
                    text.append(i)
            clean_text = ' '.join(text)
            cleantext.append(clean_text)
            insta_dict['text'].append(cleantext)
        except:
            text=[]
            insta_dict['text'].append(text)



        ##hashtag 수집
        raw_tags = self.driver.find_elements_by_css_selector('a.xil3i')
        hash_tag = []
        for i in raw_tags:
            hash_tag.append(i.text)
        insta_dict['hashtag'].append(hash_tag)
        
        
        '''c드라이브에 IG폴더 만들고 secret.txt 파일에 윗줄에 아이디 2번째줄에 비밀번호 입력'''    
if __name__=='__main__':
    "Main (runs through classes)"
    data=pd.read_csv('c:\IG\secret.txt',header=None)
    username=data.iloc[0][0]
    password=data.iloc[1][0]
    
    '''처음실행하고 쿠키저장'''
def first_login():     
    '''Check login to instagram and save cookies'''
    bot=IG_Bot(username,password)
    bot.login()
    bot.save_cookies()

    '''쿠키저장 후 크롤러 가동하기'''
def run_():
    # '''Crawler'''
    bot=Crawler(username,password)
    bot.run()
    
run_()
'''아래 셀은 쿠기 저장 예시'''

<ipython-input-1-8037d59ce9c9>:43: DeprecationWarning: use options instead of chrome_options
  self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)


검색어를 입력하시오. > 혜화역4번출구


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.C4VMK"}
  (Session info: chrome=94.0.4606.81)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.19042 x86_64)


In [3]:
len(insta_dict['img'])

443

In [4]:
with open('c:\IG\혜화역4번출구_443', 'w') as f:
    json.dump(insta_dict, f,indent=4)

In [50]:
with open('c:\IG\sample.json','r') as f:
    json_file=json.load(f)
    
print(json_file)

{'img': ['https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/244653873_1201747707022036_773658369982129180_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=zB4lAeWvoxIAX-XvovG&edm=AP_V10EBAAAA&ccb=7-4&oh=c58fedfc582476a7672ae898ff43bf96&oe=6170887E&_nc_sid=4f375e', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/244653873_1201747707022036_773658369982129180_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=zB4lAeWvoxIAX-XvovG&edm=AP_V10EBAAAA&ccb=7-4&oh=c58fedfc582476a7672ae898ff43bf96&oe=6170887E&_nc_sid=4f375e', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/244653873_1201747707022036_773658369982129180_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=zB4lAeWvoxIAX-XvovG&edm=AP_V10EBAAAA&ccb=7-4&oh=c58fedfc582476a7672ae898ff43bf96&oe=6170887E&_nc_sid=4f375e', 'https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/244653873_1201747707022036_773658369982129180_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagr

In [136]:
'''예시 아래 결과창'''

import pandas as pd
from selenium.webdriver.common.keys import Keys
def crawl(self):
    insta_dict = {'img':[],
                    'id':[],
                  'date': [],
                  'like': [],
                  'text': [],
                  'hashtag':[]}
    ## 이미지 수집

    try:
        srcs=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div/div/div/div/video')
        src=srcs.get_attribute('poster')
        insta_dirct['img'].append(src)
    except:
        src=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div/img')
        src=src.get_attribute('src')
        insta_dict['img'].append(src)
    ## id 정보 수집
    try:
        info_id = driver.find_element_by_css_selector('h2._6lAjh').text
        insta_dict['id'].append(info_id)
    except:
        info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
        insta_dict['id'].append(info_id)
    print(insta_dict['id'])
    print()


    ## 시간정보 수집 
    upload_time=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/div[2]/a/time')
    print(upload_time.get_attribute('title'))
    time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
    time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
    insta_dict['date'].append(time_info)
    print(insta_dict['date'])
    print()

    ## like 정보 수집
    try:
        check=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/span/span')
        
    except:
        try:
            like=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div/a/span')
            insta_dict['like'].append(like)
            
        except:
            id=[]
            driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div[2]/a[2]').click()
            while True:
                last_height =driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                c=driver.find_elements_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a')

                for i in c:  
                    if i.text not in id:
                        id.append(i.text)     
                    else:
                        pass
                
                driver.find_element_by_css_selector('div._1XyCr').click()
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(1)
                new_height =driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                if last_height==new_height:
                    driver.find_element_by_xpath('/html/body/div[7]/div/div/div[1]/div/div[2]/button').click()
                    break

            like=len(id)
            insta_dict['like'].append(like)

    ##text 정보수집
    raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
    raw_info = raw_info[1:len(raw_info)-1]
    text=[]
    for i in raw_info:
        if '#' in i:
            pass
        else:
            text.append(i)
    clean_text = ' '.join(text)
    text.append(clean_text)
    insta_dict['text'].append(text)



    ##hashtag 수집
    raw_tags = driver.find_elements_by_css_selector('a.xil3i')
    hash_tag = []
    for i in raw_tags:
        hash_tag.append(i.text)
    insta_dict['hashtag'].append(hash_tag)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/239218352_335498761650611_1251356593086644312_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=G78EYJ1SFPMAX_B6Pku&edm=AP_V10EBAAAA&ccb=7-4&oh=76b23c271d1ea87146c2781573ac94fe&oe=616D2144&_nc_sid=4f375e
['hello_storylibrary']

2021년 8월 20일
[Timestamp('2021-08-20 00:00:00+0000', tz='UTC')]

26

["[스토리라이브러리만의 특별한 영화 추천, 스라의 빛✨소개] 🎬 너무 좋아서 벽치다가 무너져서 🎬 옆집 만날 장르?! 🎬 말이 필요없는 망작?! 🎬 꿈에 나올것 같았던 영화?! 🎬 간단하게 보기 좋은 영화?! 🎬 꿈인지 현실인지 헷갈리는 영화?! 스라러 재영이 만들어준 영화 추천 큐레이션 게시판, '스라의 빛' 반응이 핫해요! 이미 있는 영화 장르를 제외하고 새로운 장르와 장르에 어울리는 영화를 추천하는 게시판이에요. 게시판이 생긴지 한달도 되지 않아서 어느새 빼곡히 추천 장르, 영화로 가득찼어요. 하루 하루 스라러들이 슬쩍 남겨주고 간 흔적을 읽어보는 재미가 쏠쏠해요. 새로운 영화를 살펴보며 낯선 영감을 만나고 싶다면 스라로 놀러오세요! 오늘도 스토리라이브러리 문 활-짝! 저녁 6시까지 문을 열어둘께요. 9월부터는 수, 목, 금요일은 오후 5시부터 저녁 9시까지, 주말엔 오후 12시부터 6시까지 문을 열어요! 놀러오고 싶다면 프로필 링크를 클릭👆"]

['#스토리라이브러리', '#스라', '#스라러', '#스라의빛', '#영화', '#영화추천', '#장르', '#장르추천', '#1219세전용작업실', '#혜화역2번출구']


In [43]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

url = 'https://instagram.com'

# 드라이버 연결
driver = webdriver.Chrome('c:\python_temp\chromedriver.exe')
# 웹사이트 이동
driver.get(url)
time.sleep(1)
'''Load cookies into browser'''
cookies= json.load(open('c:\IG\cookies.json','rb'))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.refresh()

In [ ]:
# 원하는 요소(element)를 찾습니다. 이 경우에는 검색창입니다.
search_box = driver.find_element_by_id('query')
# 다음과 같은 동작을 하는 액션 체인을 만들었습니다.
# 검색창을 찾고 '아이스크림'이라는 검색어를 입력한 뒤 Enter를 입력합니다.
actions = webdriver.ActionChains(driver).send_keys_to_element(search_box, '아이스크림').send_keys(Keys.ENTER)
# 체인을 실행합니다.
actions.perform()

In [44]:
try:
    info_id = driver.find_element_by_css_selector('h2._6lAjh').text
    print(info_id)
except:
    try:          
        info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
        print(info_id)
    except:  
        info_id = driver.find_element_by_css_selector('div.e1e1d').text
        print(info_id)

rrryu.my


In [82]:
with open('c:\IG\혜화역1번출구_660.json','r') as f:
    json_file=json.load(f)
    
a=sum(json_file['text'],[])
print(a)

['보고 오는길에 발견.😍😍 🐤그냥 지나칠순 없지. 😊 . (4천원) 가져오면 함께 볶아준대서 옆집 🍳 에서 순대도 구매 . 순대까지 추가 했더니 어마무시한 양이 되었어요. 빨리 집가서 해야지. ( ⛳ 다이소 맞은편 💸) 🍭😆🎶🎃🏝️👯😝😂🕺💃 그리고 🍪 뽑기집들 요즘 🦑 덕분에 아주 2번출구 납품한 뽑기집은 까지 배포하고 있어요.🤣🤣🤣 🧑\u200d🎓 🍭\U0001fa85🍬🤩 오늘의 \U0001fa85🕺🍬\U0001fa85💃🤩🎶🛍️', '코로나19가 격상된 지금.. 멘탈잡기가 쉽지 않는 요즘.. 황대표님에 추천연극.. 참기름톡! 20년12월19일..21년1월3일 까지.. 짧은 공연이라..부랴부랴 달려갔다. 코로나19 때문에.. 소중함을 잠시 잊고 있던, 나에게..큰 교훈을 주기에는 충분했다. 소중한것을..지키기 위해.. 힘내자..아자..👍 20.12.23', '우리가 사랑하는 작은 숲 . mondforêt 혜화동 로터리 한 골목 오랜 역사가 있는 곳에서 , 제가 좋아하는 꽃과 식물들을 보여 드릴 수 있는 공간을 오픈합니다 꽃을 처음 만질 때 부터 키워왔던 꿈을 좋은 기회로 이루게 되서 떨리고 설레네요.. - 🧡🌲 우리, 몬드포레에서 만나요 ! - - 종로구 창경궁로 35길 21 1F 꽃 주문 및 문의 010 2415 8040', '초대권이 생겨 보게 된 대학로 연극 예매 멀티녀 이슬기님 살앙훼욥😘 먹는 내내 광대 승천 최강 맛집 뿌수긔😝💕', '[이벤트🎉 ] - 가 준비한 를 외치는 시어터 가족들을 위해 팡팡 터지고 있는 이벤트! (오늘도 벌써 3개째...) 이번에는 9월 7일 개막을 앞두고 있는 연극 <에쿠우스>에 여러분을 초대합니다🐴 매 시즌 강렬한 무대로 관객들을 사로잡는 바로 그 작품! 올해도 놓칠 수 없는 캐스트로 찾아옵니다! ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ 🎁 인스타 팔로잉 후 [친구소환+기대평]을 댓글로 남겨주세요. 추첨을 통해 공연에 초대합니다! 리그램을 하면 당첨 확률이 UP➕ * 초대일시: 9.11(수) 20시/ 1인 2매-R석/ 현장수령 * 